In [7]:
""" 
Author : Hritik Jain
Date   : 11/12/2016

In this script, I'll produce a Dataset with the single image of a Rs.500 currency note
and its fake version, which I simulated by erasing the watermark of Mahatma Gandhi with 
an online image editor.
The aim is to teach a CNN to differenciate between a fake and a real currency note.
_______________________________________________________________________________________

I'm first going to try it out with the OpenCV library.
"""

" \nAuthor : Hritik Jain\nDate   : 11/12/2016\n\nIn this script, I'll produce a Dataset with the single image of a Rs.500 currency note\nand its fake version, which I simulated by erasing the watermark of Mahatma Gandhi with \nan online image editor.\nThe aim is to teach a CNN to differenciate between a fake and a real currency note.\n_______________________________________________________________________________________\n\nI'm first going to try it out with the OpenCV library.\n"

In [1]:
import os
import cv2
import numpy as np

In [15]:
# checking out the basic functionality

img = cv2.imread('500 Testing/New Real 4/2.jpg')
# converting to gray-scale
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
img = cv2.resize(img, (1000, 500), interpolation = cv2.INTER_AREA)
cv2.namedWindow('img', cv2.WINDOW_AUTOSIZE)
cv2.imshow('img', img)
# similarly, images can be written using the function imwrite and with the same arguments
cv2.waitKey()

(500, 1000)

In [74]:
"""
Assuming the image's name is passed as a command line argument,
here's how we will get the image in a proper format to feed into 
the classifier, first resizing it to match the input layer size
which for now let's fix at 300 x 600, aspect ratio being 2:1
"""
# wrap it into a function
# import sys
# img = cv2.imread(sys.arg[1])
# img = cv2.resize(img,(1000, 500), interpolation = cv2.INTER_AREA)

"\nAssuming the image's name is passed as a command line argument,\nhere's how we will get the image in a proper format to feed into \nthe classifier, first resizing it to match the input layer size\nwhich for now let's fix at 300 x 600, aspect ratio being 2:1\n"

In [2]:
# this will be the function used to resize and grayscale the the raw input image
def resizeAndGray(img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = cv2.resize(img, (600, 300), interpolation = cv2.INTER_AREA)
    return img

In [89]:
# let's produce the dataset

In [3]:
# for efficient memory storage :
# since the numbers are between 0 and 255, numpy's int16 datatype can be used
# secondly, we need to flatten the image array to store it efficiently ad row-major
# so for that, we will numpy's use numpy's ravel function with 'copy' flag = False
def generateDataForImages():
    """
    In this function, various techniques for augementing image data, such as random rotations
    and translations, zooming on images will performed to produce a dataset of size ~7000 images.
    RETURN :
        list of numpy arrays, one for each image
    """
    
    # this will be the function used to resize and grayscale the the raw input image
    def resizeAndGray(img):
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img = cv2.resize(img, (600, 300), interpolation = cv2.INTER_AREA)
        return img
    
    trainingInput = []
    noOfRows = 300
    noOfCols = 600
    
    # first all real currency notes' images and then all the fake ones'
    # i am skeptical of the consequences :-/
    for directory in ['500 Testing/New Real 4/', '500 Testing/New Fake 4/']:
        for filename in os.listdir(directory):
            print filename
            img = cv2.imread(directory + filename)
            img = resizeAndGray(img)

            # TRANSLATIONS
            # will produce (10 + 10 + 1) x (10 + 10 + 1) = 441 images
            # stride of 10 pixels along both axis along all 4 directions
            for x in range(100, -110, -10):
                for y in range(100, -110, -10):
                    translationMatrix = np.float32([ [1,0,x], [0,1,y] ])
                    imgTrns = cv2.warpAffine(img, translationMatrix, (noOfCols, noOfRows))
                    trainingInput.append(np.ravel(imgTrns.astype(np.int16, copy = False)))


            # ROTATIONS
            # we produce 60 different angles in the range of -30 to 30
            # with the step being equal to 0.5
            for angle in range(30, -30, -1):
                rotationMatrix = cv2.getRotationMatrix2D((noOfCols/2, noOfRows/2), float(angle)/2, 1)
                imgRotated = cv2.warpAffine(img, rotationMatrix, (noOfCols, noOfRows))
                trainingInput.append(np.ravel(imgRotated.astype(np.int16, copy = False)))

            # PROJECTIVE TRANSFORMATIONS for ZOOMING IN AND ZOOMING OUT
            # will produce (20 + 20) images for the dataset
            # 1ST ZOOMING IN ...
            for step in np.arange(0.005, 0.105, 0.005):
                srcPoints = np.float32([[int(step*(noOfCols-1)),int(step*(noOfRows-1))], [int((1-step)*(noOfCols-1)),int(step*(noOfRows-1))], [int(step*(noOfCols-1)),int((1-step)*(noOfRows-1))], [int((1-step)*(noOfCols-1)), int((1-step)*(noOfRows-1))]])
                dstPoints = np.float32([[0,0], [noOfCols-1,0], [0,noOfRows-1], [noOfCols-1,noOfRows-1]]) 
                projective_matrix = cv2.getPerspectiveTransform(srcPoints, dstPoints)
                imgZoomed = cv2.warpPerspective(img, projective_matrix, (noOfCols,noOfRows))
                trainingInput.append(np.ravel(imgZoomed.astype(np.int16, copy = False)))
            # 2ND ZOOMING OUT ...
            for step in np.arange(0.005, 0.105, 0.005):
                srcPoints = np.float32(np.float32([[0,0], [noOfCols-1,0], [0,noOfRows-1], [noOfCols-1,noOfRows-1]]))
                dstPoints = np.float32([[int(step*(noOfCols-1)),int(step*(noOfRows-1))], [int((1-step)*(noOfCols-1)),int(step*(noOfRows-1))], [int(step*(noOfCols-1)),int((1-step)*(noOfRows-1))], [int((1-step)*(noOfCols-1)), int((1-step)*(noOfRows-1))]]) 
                projective_matrix = cv2.getPerspectiveTransform(srcPoints, dstPoints)
                imgZoomed = cv2.warpPerspective(img, projective_matrix, (noOfCols,noOfRows))
                trainingInput.append(np.ravel(imgZoomed.astype(np.int16, copy = False)))

    return trainingInput

In [ ]:
trainingInput = generateDataForImages()

In [5]:
trainingOutput = [1]*(len(trainingInput)/2) + [0]*(len(trainingInput)/2)
dataset = (trainingInput, trainingOutput)

In [6]:
import cPickle
with open('trainingInput.pkl','wb') as fp:
    cPickle.dump(dataset,fp, protocol = 2)

In [10]:
# taking first 1000 images of both real and fake to see performance
smallTrainingInput = trainingInput[1:1001] + trainingInput[7034:8034]

In [12]:
smallTrainingOutput = [1]*1000 + [0]*1000

In [13]:
mallDataset = (smallTrainingInput, smallTrainingOutput)

In [14]:
# THIS FILE IS NOW MODIFIED AS PER THE CODE BELOW ! THIS WAS AN EARLY MISTAKE
with open('smallFakeCurrencyDetectionData.pkl','wb') as fp:
    cPickle.dump(smallDataset,fp, protocol = 2)

In [6]:
import cPickle
with open('smallFakeCurrencyDetectionData.pkl','rb') as fp:
    trainData, validationData, testData = cPickle.load(fp)

In [11]:
trainingInput = trainData[0][:900] + validationData[0][:50] + testData[0][:50] + trainData[0][900:] + validationData[0][50:] + testData[0][50:]

In [18]:
# here's updating the file smallFakeCurrencyDetectionData.pkl
# just trying to make it compatible with Michael Nelson code's load_data_shared() function
trainDataInput = []
for i in xrange(0,900):
    trainDataInput.append(trainingInput[i]) 
    trainDataInput.append(trainingInput[1000+i])

validationDataInput = trainingInput[900:950] + trainingInput[1900:1950]
testDataInput = trainingInput[950:1000] + trainingInput[1950:1999]

trainDataOutput = []
for i in xrange(900):
    trainDataOutput += [1,0]
    
validationDataOutput = [1]*50 + [0]*50
testDataOutput = [1]*50 + [0]*49

trainData = (trainDataInput, trainDataOutput)
validationData = (validationDataInput, validationDataOutput)
testData = (testDataInput, testDataOutput)

smallDataset = (trainData, validationData, testData)
with open('smallFakeCurrencyDetectionData.pkl','wb') as fp:
    cPickle.dump(smallDataset,fp, protocol = 2)